In [1]:
from sklearn.model_selection import KFold
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall, Metric

from dvclive import Live  # Ensure DVCLive is imported
from dvclive.keras import DVCLiveCallback  # Import the callback
import yaml

In [2]:
MAIN_PATH = os.path.dirname(os.getcwd())
DATA_PATH = MAIN_PATH + "/data/numpy"
MODEL_PATH = MAIN_PATH + "/models"
LOG_PATH = MAIN_PATH + "/logs"

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1024
NUM_FOLDS = 5

BEST_VAL_SCORE = 0
BEST_MODEL = None
HISTORY = []  # Initialize history_list


In [3]:
def load_config(config_path=MAIN_PATH + "/params.yaml"):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config()  # This loads the configuration once and allows direct access

In [4]:
def plot_history_metrics(history_dict: dict):
    total_plots = len(history_dict)
    cols = total_plots // 2
    rows = total_plots // cols
    if total_plots % cols != 0:
        rows += 1

    pos = range(1, total_plots + 1)
    plt.figure(figsize=(15, 10))
    for i, (key, value) in enumerate(history_dict.items()):
        plt.subplot(rows, cols, pos[i])
        plt.plot(range(len(value)), value)
        plt.title(str(key))
    plt.show()

In [5]:
def load_df():
    df = pd.read_csv(MAIN_PATH + "/data/result_df.csv")
    return df

In [6]:
def Clean_missing_values(numpy_data):
    numpy_data['x_train'], numpy_data['y_train'] = Remove_missing_values(numpy_data['x_train'], numpy_data['y_train'])
    numpy_data['x_val'], numpy_data['y_val'] = Remove_missing_values(numpy_data['x_val'], numpy_data['y_val'])
    numpy_data['x_test_1'], numpy_data['y_test_1'] = Remove_missing_values(numpy_data['x_test_1'], numpy_data['y_test_1'])
    numpy_data['x_test_2'], numpy_data['y_test_2'] = Remove_missing_values(numpy_data['x_test_2'], numpy_data['y_test_2'])
    
    return numpy_data

def Remove_missing_values(x_data, y_data):
    # Check if y_data contains missing values (NaNs) and remove corresponding x_data rows
    valid_indices = ~np.isnan(y_data)  # Find valid (non-NaN) indices in y_data
    x_clean = x_data[valid_indices]
    y_clean = y_data[valid_indices]
    return x_clean, y_clean

In [7]:
def gather_numpy_files(data_path):
    numpy_data = {}
    
    for file_name in os.listdir(data_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(data_path, file_name)
            numpy_data[file_name[:-4]] = np.load(file_path)  # Store in dict
    
    # Clean data by removing rows where y_* contains missing values
    numpy_data = Clean_missing_values(numpy_data)

    return numpy_data

In [8]:
def calculate_class_weights(df, label_column):
    vals_dict = {}
    for i in df[label_column]:
        if i in vals_dict.keys():
            vals_dict[i] += 1
        else:
            vals_dict[i] = 1
    total = sum(vals_dict.values())
    weight_dict = {k: (1 - (v / total)) for k, v in vals_dict.items()}

    print(f"Weight dict for model: {weight_dict}")
    return weight_dict

In [9]:
def create_model():
    input_layer = keras.Input(shape=(config["model"]["input_shape"], config["model"]["input_features"]))
    
    x = layers.Conv1D(filters=32, kernel_size=config["model"]["kernel_size"], activation=config["model"]["activation"], padding="same", kernel_regularizer=regularizers.l2(0.001))(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(filters=64, kernel_size=3, activation=config["model"]["activation"], padding="same", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(filters=128, kernel_size=3, activation=config["model"]["activation"], padding="same", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    
    output_layer = layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    
    return model


In [10]:
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

In [11]:
def Compile_model():
    model = create_model()
    optimizer = keras.optimizers.Adam(amsgrad=True, learning_rate=config["model"]["learning_rate"])
    loss = keras.losses.BinaryCrossentropy()
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            keras.metrics.BinaryAccuracy(name='binary_accuracy'),
            keras.metrics.AUC(name='auc'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            F1Score(name='f1_score')
        ],
    )
    return model

In [12]:
def SplitDatasetForFolds(train_index, validation_index, fold_nr, numpy_data):
    print(f"Training fold {fold_nr}...")

    # Split the data into train sets for this fold.
    x_train_fold = numpy_data['x_train'][train_index]
    y_train_fold = numpy_data['y_train'][train_index]

    print(f"x_val shape: {numpy_data['x_val'].shape}")
    print(f"y_val shape: {numpy_data['y_val'].shape}")
    
    # Ensure to use only the training set indices
    x_validation_fold = numpy_data['x_val'][:len(validation_index)]  # Taking the first `len(validation_index)` samples
    y_validation_fold = numpy_data['y_val'][:len(validation_index)]

    # Create tf.data.Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_fold, y_train_fold))
    validation_dataset = tf.data.Dataset.from_tensor_slices((x_validation_fold, y_validation_fold))
    test_dataset_subject1 = tf.data.Dataset.from_tensor_slices((numpy_data['x_test_1'], numpy_data['y_test_1']))	
    test_dataset_subject2 = tf.data.Dataset.from_tensor_slices((numpy_data['x_test_2'], numpy_data['y_test_2']))
    
    # Shuffling and batching the datasets
    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    validation_dataset = validation_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_dataset_subject1 = test_dataset_subject1.batch(BATCH_SIZE)
    test_dataset_subject2 = test_dataset_subject2.batch(BATCH_SIZE)

    return train_dataset, validation_dataset, test_dataset_subject1, test_dataset_subject2


In [13]:
def convert_to_native(data):
    """Recursively convert numpy types to native python types."""
    if isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    elif isinstance(data, np.ndarray):
        return data.tolist()  # Convert numpy array to list
    elif isinstance(data, (np.float32, np.float64)):
        return data.item()  # Convert single value numpy float to Python float
    else:
        return data  

In [14]:
def save_history_to_json(history, fold_number, best_model):
    # Create a dictionary for the current fold's metrics
    metrics = {
        "fold_number": fold_number,
        "val_accuracy": history.history['val_accuracy'][-1],
        "val_loss": history.history['val_loss'][-1],
        "best_model": best_model
    }

    # Load existing metrics if the file exists
    if os.path.exists('metrics.json'):
        with open('metrics.json', 'r') as f:
            existing_metrics = json.load(f)
    else:
        existing_metrics = []

    # Append the new metrics
    existing_metrics.append(metrics)

    # Write back the updated metrics to the file
    with open('metrics.json', 'w') as f:
        json.dump(existing_metrics, f, indent=4)

In [15]:
def Train_fold(train_index, val_index, fold_number, numpy_data, weight_dict, live):
    global BEST_VAL_SCORE, HISTORY 

    # Split data into training and validation sets for this fold
    train_dataset, validation_dataset, test_sj1, test_sj2 = SplitDatasetForFolds(train_index, val_index, fold_number, numpy_data)

    # Create and compile the model
    model = Compile_model()

    # Set up DVC Live callback for this fold
    live_callback = DVCLiveCallback()  # No need to pass live here

    # Set up other callbacks
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            os.path.join(MODEL_PATH, f"best_model_fold_{fold_number}.keras"),
            save_best_only=True,
            monitor="val_accuracy"
        ),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001),
        live_callback  # Add DVCLiveCallback to the list
    ]

    print("Starting training...")
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=config['model']['epochs'],
        validation_data=(validation_dataset),
        callbacks=callbacks,
        class_weight=weight_dict
    )

    # Create models directory if it doesn't exist
    os.makedirs(MODEL_PATH, exist_ok=True)
    
    # Save the model to the models directory
    model.save(os.path.join(MODEL_PATH, f'model_{fold_number}.h5'))
    print(f'Model saved to models/model_{fold_number}.h5')

    print(f"Training fold {fold_number} completed\n")


In [16]:
def Cross_validation_training(numpy_data, weight_dict):
    scores = []
    os.makedirs(MODEL_PATH, exist_ok=True)  # Ensure the model path exists

    # Initialize KFold with the number of splits
    kfold = KFold(n_splits=config['model']['folds'], shuffle=True, random_state=42)

    try:
        for fold_number, (train_index, val_index) in enumerate(kfold.split(numpy_data['x_train']), start=1):
            print(f"Training fold {fold_number}")
            
            # Modify the experiment name to a simpler format without "branch" or underscores
            exp_name = f"fold-{fold_number}".lower()

            # Create a Live instance with the updated experiment name
            live = Live(exp_name=exp_name,
                        exp_message=f"Training fold {fold_number}")

            # Train the current fold and calculate score
            score = Train_fold(train_index, val_index, fold_number, numpy_data, weight_dict, live)
            scores.append(score)

            live.end()  # End the live logging for this fold

    except Exception as e:
        print(f"An error occurred during cross-validation training: {e}")

    return scores

In [17]:
def main():
    df = load_df();
    numpy_data = gather_numpy_files(DATA_PATH)

    # Calculate weights
    weight_dict = calculate_class_weights(df, 'downsampled_label')

    # Create model
    convolutional_model = create_model()
    convolutional_model.summary()

    # Train model
    Cross_validation_training(numpy_data, weight_dict)


    return numpy_data;

In [ ]:
numpy = main()

Weight dict for model: {0: 0.11450662739322537, 1: 0.8854933726067746}
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 32, 32)            128       
                                                                 
 batch_normalization (BatchN  (None, 32, 32)           128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 64)            6208      
                                                        

c:\Users\goert\.conda\envs\tf\lib\site-packages\keras\engine\training.py:2319: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


244/244 [==============================] - 16s 35ms/step - loss: 0.8572 - binary_accuracy: 0.7705 - auc: 0.9225 - precision: 0.7006 - recall: 0.9485 - f1_score: 0.8059 - val_loss: 3.6429 - val_binary_accuracy: 0.3612 - val_auc: 0.5484 - val_precision: 0.3612 - val_recall: 1.0000 - val_f1_score: 0.5307 - lr: 0.0010
Epoch 2/3
244/244 [==============================] - 8s 34ms/step - loss: 0.5049 - binary_accuracy: 0.7724 - auc: 0.9299 - precision: 0.6979 - recall: 0.9646 - f1_score: 0.8098 - val_loss: 2.3251 - val_binary_accuracy: 0.3645 - val_auc: 0.7744 - val_precision: 0.3624 - val_recall: 1.0000 - val_f1_score: 0.5320 - lr: 0.0010
Epoch 3/3
244/244 [==============================] - 8s 33ms/step - loss: 0.3200 - binary_accuracy: 0.7746 - auc: 0.9341 - precision: 0.6983 - recall: 0.9707 - f1_score: 0.8123 - val_loss: 2.9535 - val_binary_accuracy: 0.3919 - val_auc: 0.7558 - val_precision: 0.3726 - val_recall: 0.9993 - val_f1_score: 0.5428 - lr: 0.0010


    dvc remote add -d <remote name> <remote url>
	config.yaml


Model saved to models/model_1.h5
Training fold 1 completed

Fold 1 metrics logged.
Training fold 2
Training fold 2...
x_val shape: (4988, 32, 1)
y_val shape: (4988,)
Starting training...
Epoch 1/3
244/244 [==============================] - 12s 34ms/step - loss: 0.8706 - binary_accuracy: 0.7675 - auc: 0.9158 - precision: 0.6945 - recall: 0.9533 - f1_score: 0.8036 - val_loss: 4.0194 - val_binary_accuracy: 0.3612 - val_auc: 0.5361 - val_precision: 0.3612 - val_recall: 1.0000 - val_f1_score: 0.5307 - lr: 0.0010
Epoch 2/3
244/244 [==============================] - 8s 33ms/step - loss: 0.5354 - binary_accuracy: 0.7608 - auc: 0.9249 - precision: 0.6839 - recall: 0.9674 - f1_score: 0.8013 - val_loss: 3.7211 - val_binary_accuracy: 0.3612 - val_auc: 0.6796 - val_precision: 0.3612 - val_recall: 1.0000 - val_f1_score: 0.5307 - lr: 0.0010
Epoch 3/3
244/244 [==============================] - 8s 35ms/step - loss: 0.3457 - binary_accuracy: 0.7570 - auc: 0.9285 - precision: 0.6796 - recall: 0.9702 - f1